# API 

In [234]:
import requests
import json
import pprint
import time
import csv

## Json

JSON 解析库并不是把 JSON 数据解析为 JSON 对象或 者 JSON 节点，而是把 JSON 数据转换为字典 ， 
JSON 数组转 换成列 表 ， JSON 字符串转换 为 Python字符串，这样便可以轻松地对 JSON数据进行操作了。

In [9]:
import json

jsonstring = '{"user_man": [{"name": "Peter"}, {"name": "xiaoming"}], '\
'"user_woman": [{"name": "Mary"}, {"name": "xiaohong"}]}'

json_data = json.loads(jsonstring)
print(json_data["user_man"][0]["name"])

Peter


## API 

调用https://rapidapi.com/ 的 `MyMemory` 获取翻译结果



In [19]:
url = "https://translated-mymemory---translation-memory.p.rapidapi.com/api/get"

querystring = {"mt": "0", "onlyprivate": "0",
               "langpair": "zh-cn|en", "q": "大象"}

headers = {
    'x-rapidapi-host': "translated-mymemory---translation-memory.p.rapidapi.com",
    'x-rapidapi-key': "ad0f12f18amsh0bfd615fab82b74p17b1d1jsn0491771acafa"
}

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{"responseData":{"translatedText":"Elephant","match":1},"quotaFinished":false,"mtLangSupported":null,"responseDetails":"","responseStatus":200,"responderId":"60","exception_code":null,"matches":[{"id":"424289961","segment":"\u5927\u8c61","translation":"Elephant","quality":"80","reference":null,"usage-count":8,"subject":"All","created-by":"Wikipedia","last-updated-by":"Wikipedia","create-date":"2012-02-11 01:13:20","last-update-date":"2012-02-11 01:13:20","match":1},{"id":"463490305","segment":"\u5927\u8c61","translation":"Elephants","quality":"74","reference":null,"usage-count":3,"subject":"All","created-by":"MateCat","last-updated-by":"MateCat","create-date":"2017-10-08 19:58:40","last-update-date":"2017-10-08 19:58:40","match":0.99},{"id":"463713069","segment":"\u5927\u8c61\u3001\u7280\u725b\u3001\u6cb3\u9a6c\uff1b","translation":"Elephant, rhinoceros, hippopotamus;","quality":"0","reference":null,"usage-count":1,"subject":"All","created-by":"MateCat","last-updated-by":"MateCat","cre

In [20]:
json_data = json.loads(response.text) 
pprint.pprint(json_data)

{'exception_code': None,
 'matches': [{'create-date': '2012-02-11 01:13:20',
              'created-by': 'Wikipedia',
              'id': '424289961',
              'last-update-date': '2012-02-11 01:13:20',
              'last-updated-by': 'Wikipedia',
              'match': 1,
              'quality': '80',
              'reference': None,
              'segment': '大象',
              'subject': 'All',
              'translation': 'Elephant',
              'usage-count': 8},
             {'create-date': '2017-10-08 19:58:40',
              'created-by': 'MateCat',
              'id': '463490305',
              'last-update-date': '2017-10-08 19:58:40',
              'last-updated-by': 'MateCat',
              'match': 0.99,
              'quality': '74',
              'reference': None,
              'segment': '大象',
              'subject': 'All',
              'translation': 'Elephants',
              'usage-count': 3},
             {'create-date': '2015-01-19 13:08:10',
           

In [18]:
json_data['matches'][0]["translation"]

'Elephant'

## 调用 Google geocoding 


In [26]:
url = "https://google-maps-geocoding.p.rapidapi.com/geocode/json"

querystring = {"language": "zh-cn",
               "address": "北京天安门"}

headers = {
    'x-rapidapi-host': "google-maps-geocoding.p.rapidapi.com",
    'x-rapidapi-key': "ad0f12f18amsh0bfd615fab82b74p17b1d1jsn0491771acafa"
}

response = requests.request("GET", url, headers=headers, params=querystring)


json_data = json.loads(response.text)
pprint.pprint(json_data)

{'results': [{'address_components': [{'long_name': '东城区',
                                      'short_name': '东城区',
                                      'types': ['political',
                                                'sublocality',
                                                'sublocality_level_1']},
                                     {'long_name': '北京市',
                                      'short_name': '北京市',
                                      'types': ['administrative_area_level_1',
                                                'political']},
                                     {'long_name': '中国',
                                      'short_name': 'CN',
                                      'types': ['country', 'political']}],
              'formatted_address': '中国北京市东城区',
              'geometry': {'location': {'lat': 39.9087202, 'lng': 116.3974799},
                           'location_type': 'GEOMETRIC_CENTER',
                           'viewport': {'north

## 爬 sydneytoday.com 的房源位置信息


In [285]:
import requests
from lxml import etree
from bs4 import BeautifulSoup
import re


headers = {
    'user-agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36"
} 

file = open("/Users/apple/Downloads/Code/crawler/todaySydney.csv",'wt', newline='', encoding= 'utf8')
writer = csv.writer(file)
writer.writerow(("lat","lng"))


9

In [286]:

for i in range(2,3):
    url = 'https://www.sydneytoday.com/house_rent-ha0-fs0-hx0-so0-jg0-cw0-p' + str(i)
    res = requests.get(url, headers=headers)
    if res.status_code == 200:
        
        if i == 1:
            soup = BeautifulSoup(res.text, "lxml")
            selector = "body > div.main.container > div > div.col-mtg-7 > div.yp-list > div > div > div.col-xs-9.no-padding-left > a"
            links = [ x.get("href") for x in soup.select(selector)]
            for link in links:
                if get_location(link) != None:
                    writer.writerow(get_location(link))
                
        else:
            json_data = json.loads(res.text) 
            for row in json_data["data"]['rows']:
                if row["_id"] != 0:
                    link = '//www.sydneytoday.com/house_rent/' + row["_id"]
                    if get_location(link) != None:
                        writer.writerow(get_location(link))
                    
        
    print(i)
    time.sleep(1)
    
    
    
file.close()

2


In [273]:
link = "//www.sydneytoday.com/house_rent/191932560495034"
get_location(link)


In [263]:
def get_location(link):
    link = "https:" + link
    res = requests.get(link, headers=headers)

    soup = BeautifulSoup(res.text, "lxml")
    try:
        address = soup.select("body > div.main.container > div > div.col-mtg-7 > div.yp-content > div.padding20 > div.row > div:nth-child(1) > div:nth-child(6)")[0].text[4:]
    except:
        pass
    else:
        print(geocoding(address))
        return geocoding(address)

    

In [244]:
def geocoding(address):
    url = "https://google-maps-geocoding.p.rapidapi.com/geocode/json"

    querystring = {"bounds":"-34.0,151.2|-33.5,150.0",
                   "language": "en",
                   "address": address}

    headers = {
        'x-rapidapi-host': "google-maps-geocoding.p.rapidapi.com",
        'x-rapidapi-key': "ad0f12f18amsh0bfd615fab82b74p17b1d1jsn0491771acafa"
    }

    response = requests.request("GET", url, headers=headers, params=querystring)


    json_data = json.loads(response.text)
    lat = json_data["results"][0]["geometry"]["location"]['lat']
    lng = json_data["results"][0]["geometry"]["location"]['lng']
    
    return lat, lng